# <div align="center">Spotify Playlist Generator </div>

## Objective

The purpose of this project was to create a spotify playlist 


### Libraries

In [117]:
# Spotipy / Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

# Data Analytics
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning
from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [118]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

cid = '362ab65e27f44f56b2bd2b2f62987e12'
secret = '3a3c7171323649d0a3f3e2b8e57283ab'
username = 'maelstromcarnage'

scope = 'user-top-read user-library-read playlist-modify-public playlist-read-private'

redirect_uri='http://localhost:8080/callback/'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [119]:
# import pandas as pd
# import numpy as np

# results = sp.current_user_top_tracks(limit=50, offset=0,time_range='long_term')

# artists = []
# albums = []
# track_names = []
# track_ids = []
# durations = []
# popularity = []

# for i, item in enumerate(results['items']):
#     artists.append(item['artists'][0]['name'])
#     albums.append(item['album']['name'])
#     track_names.append(item['name'])
#     track_ids.append(item['id'])
#     durations.append(item['duration_ms'])
#     popularity.append(item['popularity'])
    
# df_topfifty = pd.DataFrame({
#     "Track Name": track_names,
#     "Track ID": track_ids,
#     "Artist": artists,
#     "Album": albums,
#     "Duration [hh:mm:ss]": pd.to_timedelta(durations, unit='ms').astype(str).map(lambda x: x[7:]),
#     "Popularity": popularity
# })

# df_topfifty.head()

In [120]:
# features = []
# for i in range(0,len(track_names)):
#     audio_features = sp.audio_features(track_ids[i])
#     for track in audio_features:
#         features.append(track)

# df_topfifty_features = pd.DataFrame(features, index = track_names)

# # Mode (major or minor key) and type is an unnecessary feature for our analysis. Therefore we are dropping them. 
# df_topfifty_features.drop(['mode', 'type'], axis=1, inplace=True)

# df_topfifty_features.head()

In [121]:
# df_topfifty_features.hist(figsize=(40,30))
# plt.show()

In [122]:
# sns.pairplot(df_topfifty_features)

In [123]:
def user_playlist(username):
    playlist_id = []
    playlist_name = []
    num_tracks = []
    
    featured = sp.user_playlists(username)
    for i, items in enumerate(featured['items']):
        playlist_id.append(items['id'])
        playlist_name.append(items['name'])
        num_tracks.append(items['tracks']['total'])
        
    df_your_lib = pd.DataFrame({'Playlist IDs': playlist_id, 'Playlist Name': playlist_name, '# Tracks': num_tracks})
    
    return df_your_lib

In [124]:
df_user_playlists = user_playlist(username)
df_user_playlists = df_user_playlists[~df_user_playlists['Playlist Name']
                                      .str
                                      .contains('|'.join(['Custom', 'Discover']))]

# Removing playlists with less than 30 songs
df_user_playlists.drop(df_user_playlists[df_user_playlists['# Tracks']<30]
                      .index
                      , inplace = True)

df_user_playlists

,Playlist IDs,Playlist Name,# Tracks
2,37i9dQZF1Etqmy3cR2SQKX,Your Top Songs 2019,100
3,4GwcudpBXsbCmI9N6uKCJA,playlist thats definitely made by 39daph,1208
4,7429Z8tLqdozs9XR5TH2W0,Jazz Beats,251
5,3vJqJqOIyPDkX3gWfNCJbS,Sleep/Lofi/Chill Playlist,698
6,37i9dQZF1DX5g856aiKiDS,Dalkom Cafe,50
9,1kLp8FXc4tidYIRwA5qjTY,Studio Ghibli,246
10,1xY6msLHX1W34EzB0UkkbU,Guardians of the Galaxy – The Complete Mixtape,52
12,37i9dQZF1DWWwaxRea1LWS,Throwback Thursday,97
13,5B0iYhG3tY3EXKCiYKtjxW,Alternative Rock,141
15,3j1yvDP45Gsv0Y3tfDT5MC,Liked from Radio,35


In [125]:
def get_playlist_tracks(username, sourcePlaylistID):
    
    sourcePlaylist = sp.user_playlist(username, sourcePlaylistID)
    tracks = sourcePlaylist['tracks']
    songs = tracks['items']
    
    track_ids = []
    track_names = []
    
    for i in range(0, len(songs)):
        # Ensuring local tracks are not included (no data on tracks)
        if songs[i]['track']['id'] != None:
            track_ids.append(songs[i]['track']['id'])
            track_names.append(songs[i]['track']['name'])

    features = []
    for i in range(0, len(track_ids)):
        audio_features = sp.audio_features(track_ids[i])
        for track in audio_features:
            features.append(track)
    
    playlist_df = pd.DataFrame(features, index=track_names)
    
    # Returning only features that are needed
    return playlist_df[['id', 'acousticness', 'danceability', 
                        'energy', 'instrumentalness', 
                        'key', 'liveness', 'loudness', 'mode', 
                        'speechiness', 'tempo', 'valence']]
    

In [126]:
get_playlist_tracks(username, '37i9dQZF1DX5g856aiKiDS').head()

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
My Lips Like Warm Coffee,7j59Opz7qhb2aj7cnu7iLr,0.749,0.691,0.709,0.000042,4,0.116,-5.692,1,0.0799,88.050,0.244
Kangaroo,1Oi9J5OpJmnZusVBbt2cjo,0.832,0.802,0.346,0.000004,5,0.505,-6.992,0,0.0578,86.984,0.538
I wanna go out,75C9zW7Liktu6ULymEuL2y,0.750,0.587,0.440,0.000000,5,0.360,-5.623,0,0.0374,175.842,0.526
Photograph,0PkpRtJqrwuXhbdtJuQm7E,0.809,0.597,0.458,0.000000,3,0.105,-8.295,1,0.0609,90.150,0.184
"Can I Love ? (feat. youra, Meego)",4T2cOfemKB0owJS2JOu7dF,0.555,0.708,0.441,0.000005,5,0.121,-4.212,0,0.0623,79.958,0.275


In [127]:
def mean_audio_features(playlist_id):
    playlist_audio_features = get_playlist_tracks(username, playlist_id)
    return pd.DataFrame(playlist_audio_features.mean(),columns=[playlist_id])

In [128]:
user_playlists_features = []

for ids in df_user_playlists['Playlist IDs']:
    user_playlists_features.append(mean_audio_features(ids))

# user_playlists_features

In [129]:
df_user_playlists_feat = reduce(lambda left, right: pd.merge(left,right, left_index=True, right_index=True), user_playlists_features)

df_user_playlists_feat

,37i9dQZF1Etqmy3cR2SQKX,4GwcudpBXsbCmI9N6uKCJA,7429Z8tLqdozs9XR5TH2W0,3vJqJqOIyPDkX3gWfNCJbS,37i9dQZF1DX5g856aiKiDS,1kLp8FXc4tidYIRwA5qjTY,1xY6msLHX1W34EzB0UkkbU,37i9dQZF1DWWwaxRea1LWS,5B0iYhG3tY3EXKCiYKtjxW,3j1yvDP45Gsv0Y3tfDT5MC
acousticness,0.346001,0.441738,0.239492,0.626588,0.581350,0.907560,0.351162,0.118994,0.070749,0.041106
danceability,0.619010,0.666680,0.704140,0.693250,0.679280,0.340751,0.477810,0.694773,0.530720,0.560400
energy,0.547595,0.493740,0.498040,0.298370,0.495000,0.192439,0.589188,0.731443,0.766000,0.785429
instrumentalness,0.154603,0.049592,0.694038,0.705620,0.037718,0.734011,0.290390,0.043800,0.063948,0.028385
key,5.560000,5.290000,5.590000,5.700000,6.140000,4.130000,5.384615,5.422680,5.360000,5.200000
liveness,0.189125,0.182684,0.164820,0.163213,0.169086,0.153916,0.203094,0.183635,0.181569,0.201523
loudness,-9.584010,-8.718860,-9.266550,-14.582730,-7.436320,-17.661700,-11.297058,-7.408546,-5.648570,-5.252800
mode,0.590000,0.610000,0.500000,0.590000,0.480000,0.590000,0.711538,0.659794,0.720000,0.771429
speechiness,0.121716,0.107282,0.086024,0.093265,0.089686,0.042646,0.058562,0.060119,0.053364,0.099643
tempo,117.187820,109.832880,107.558400,110.249170,106.501460,105.038330,115.657712,120.298309,126.579550,129.920857


In [130]:
mean_audio_features('37i9dQZF1DX5g856aiKiDS')

,37i9dQZF1DX5g856aiKiDS
acousticness,0.581350
danceability,0.679280
energy,0.495000
instrumentalness,0.037718
key,6.140000
liveness,0.169086
loudness,-7.436320
mode,0.480000
speechiness,0.089686
tempo,106.501460


In [131]:
playlist_tracks_df = get_playlist_tracks(username, '37i9dQZF1DX5g856aiKiDS')
playlist_tracks_df


,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
My Lips Like Warm Coffee,7j59Opz7qhb2aj7cnu7iLr,0.7490,0.691,0.709,0.000042,4,0.1160,-5.692,1,0.0799,88.050,0.2440
Kangaroo,1Oi9J5OpJmnZusVBbt2cjo,0.8320,0.802,0.346,0.000004,5,0.5050,-6.992,0,0.0578,86.984,0.5380
I wanna go out,75C9zW7Liktu6ULymEuL2y,0.7500,0.587,0.440,0.000000,5,0.3600,-5.623,0,0.0374,175.842,0.5260
Photograph,0PkpRtJqrwuXhbdtJuQm7E,0.8090,0.597,0.458,0.000000,3,0.1050,-8.295,1,0.0609,90.150,0.1840
"Can I Love ? (feat. youra, Meego)",4T2cOfemKB0owJS2JOu7dF,0.5550,0.708,0.441,0.000005,5,0.1210,-4.212,0,0.0623,79.958,0.2750
Like the Movie 그 드라마처럼,1IPWWbMswU3M7ACApQaVF0,0.5970,0.810,0.310,0.000016,5,0.0906,-6.707,0,0.0591,127.910,0.5000
4hours,6VqTWy8VqZhYVrUz0foEl2,0.3360,0.442,0.628,0.000013,8,0.1330,-5.915,1,0.0777,173.917,0.5940
Mirror,4Pu5fPc24pPEdGFVspLTrV,0.7130,0.843,0.519,0.001070,11,0.2340,-7.541,0,0.1200,78.999,0.6320
Good Place,4c906O5t7hKcRutzsOyJE0,0.6270,0.708,0.768,0.000325,7,0.0771,-7.429,0,0.2610,105.206,0.4670
Winter Blossom (Prod. by 0channel),5BVjsCcDlYgJG2bB2jsOeI,0.5530,0.670,0.532,0.000349,5,0.0956,-4.661,0,0.2540,159.948,0.7180


In [132]:
def get_recommendations(tracks):
    recommendations = []
    for x in tracks:
        # Spotify sometimes has songs unavailable which we can't get recommendations for
        try:
            data = sp.recommendations(seed_tracks=[x], limit=5)
            recommendations.extend(data['tracks'])
        except:
            pass
    return recommendations

In [133]:
df_user_playlists['Playlist Name']

2                                Your Top Songs 2019
3           playlist thats definitely made by 39daph
4                                         Jazz Beats
5                          Sleep/Lofi/Chill Playlist
6                                        Dalkom Cafe
9                                      Studio Ghibli
10    Guardians of the Galaxy – The Complete Mixtape
12                                Throwback Thursday
13                                  Alternative Rock
15                                  Liked from Radio
Name: Playlist Name, dtype: object

In [134]:
recommendations = get_recommendations(playlist_tracks_df['id'])
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.head()


,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,194173,False,{'isrc': 'KRMIM2121975'},{'spotify': 'https://open.spotify.com/track/2Z...,https://api.spotify.com/v1/tracks/2ZEJPpvO2j1K...,2ZEJPpvO2j1KDx9UxShxID,False,CARDIGAN (feat. Wonstein),52,https://p.scdn.co/mp3-preview/de1c8942ad640feb...,2,track,spotify:track:2ZEJPpvO2j1KDx9UxShxID
1,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,303188,False,{'isrc': 'KRB462100205'},{'spotify': 'https://open.spotify.com/track/1n...,https://api.spotify.com/v1/tracks/1ncOOwpA56YH...,1ncOOwpA56YHuulS9xIl62,False,Sunny,45,https://p.scdn.co/mp3-preview/0f927314fd07db5f...,1,track,spotify:track:1ncOOwpA56YHuulS9xIl62
2,"{'album_type': 'COMPILATION', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,[],1,167707,False,{'isrc': 'UK57K1601420'},{'spotify': 'https://open.spotify.com/track/3c...,https://api.spotify.com/v1/tracks/3cVJ8o4oLAHw...,3cVJ8o4oLAHwLe8PTZZyDg,False,가슴 난로 Heart Stove,0,None,9,track,spotify:track:3cVJ8o4oLAHwLe8PTZZyDg
3,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,214133,False,{'isrc': 'QZF452086999'},{'spotify': 'https://open.spotify.com/track/4U...,https://api.spotify.com/v1/tracks/4UpsCID8dfHq...,4UpsCID8dfHqO4dsr3jk7k,False,My Lips Like Warm Coffee - Instrumental,36,https://p.scdn.co/mp3-preview/e621820eb6b94c3f...,2,track,spotify:track:4UpsCID8dfHqO4dsr3jk7k
4,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,195133,False,{'isrc': 'KRG132100003'},{'spotify': 'https://open.spotify.com/track/3F...,https://api.spotify.com/v1/tracks/3Flz3eI2I7Td...,3Flz3eI2I7TdDXXEm3Lp9a,False,525,40,https://p.scdn.co/mp3-preview/cbc95f23dbd76b9b...,3,track,spotify:track:3Flz3eI2I7TdDXXEm3Lp9a


In [135]:
def get_audio_features_df(df):
    
    features = []
    track_id = []
    
    for song_id in df['id']:
        audio_features = sp.audio_features(song_id)
        for track in audio_features:
            features.append(track)
    
    playlist_df = pd.DataFrame(features)
    
    # Returning only features that are needed
    return playlist_df[['id', 'acousticness', 'danceability', 
                        'energy', 'instrumentalness', 
                        'key', 'liveness', 'loudness', 'mode', 
                        'speechiness', 'tempo', 'valence']]

In [136]:
recommendations_df = get_audio_features_df(recommendations_df)
recommendations_df

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,2ZEJPpvO2j1KDx9UxShxID,0.241,0.752,0.939,0.000000,3,0.1810,-2.790,0,0.1210,100.023,0.794
1,1ncOOwpA56YHuulS9xIl62,0.590,0.452,0.425,0.000067,5,0.3730,-10.202,0,0.0823,83.721,0.273
2,3cVJ8o4oLAHwLe8PTZZyDg,0.994,0.637,0.144,0.918000,2,0.0752,-20.227,0,0.0838,165.938,0.431
3,4UpsCID8dfHqO4dsr3jk7k,0.660,0.668,0.574,0.932000,4,0.1140,-7.997,1,0.0396,88.048,0.179
4,3Flz3eI2I7TdDXXEm3Lp9a,0.264,0.709,0.827,0.000000,0,0.2890,-2.071,0,0.0650,131.951,0.561
...,...,...,...,...,...,...,...,...,...,...,...,...
245,7LsEAHFG596DGoceluRylu,0.895,0.592,0.317,0.847000,6,0.4080,-12.983,1,0.0864,84.965,0.533
246,1SPDWTBH7qcjbZ8zMRXlQ9,0.722,0.797,0.386,0.000000,2,0.1470,-6.597,1,0.0301,97.942,0.530
247,5gNwUVmtSSxrM1TyvGJ9kn,0.793,0.580,0.635,0.000000,3,0.3700,-6.068,1,0.0448,92.000,0.154
248,2z20uLaRDT3Z9oD9Uo8iqv,0.778,0.599,0.219,0.000003,4,0.1970,-10.771,1,0.0385,108.919,0.308


In [137]:
playlist_tracks_df['ratings'] = [10, 6, 7, 7, 8, 9, 8, 9, 4, 9,
                                4, 6, 10, 9, 7, 4, 10, 7, 8, 10,
                                6, 5, 8, 9, 4, 7, 7, 4, 4, 6,
                                8, 7, 4, 4, 6, 6, 6, 6, 8, 5,
                                5, 7, 7, 8, 9, 8, 7, 8, 9, 8]

playlist_tracks_df.head()

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,ratings
My Lips Like Warm Coffee,7j59Opz7qhb2aj7cnu7iLr,0.749,0.691,0.709,0.000042,4,0.116,-5.692,1,0.0799,88.050,0.244,10
Kangaroo,1Oi9J5OpJmnZusVBbt2cjo,0.832,0.802,0.346,0.000004,5,0.505,-6.992,0,0.0578,86.984,0.538,6
I wanna go out,75C9zW7Liktu6ULymEuL2y,0.750,0.587,0.440,0.000000,5,0.360,-5.623,0,0.0374,175.842,0.526,7
Photograph,0PkpRtJqrwuXhbdtJuQm7E,0.809,0.597,0.458,0.000000,3,0.105,-8.295,1,0.0609,90.150,0.184,7
"Can I Love ? (feat. youra, Meego)",4T2cOfemKB0owJS2JOu7dF,0.555,0.708,0.441,0.000005,5,0.121,-4.212,0,0.0623,79.958,0.275,8


In [138]:
X_train = playlist_tracks_df.drop(['id', 'ratings', 'mode'], axis=1)
y_train = playlist_tracks_df['ratings']

In [139]:
X_scaled = StandardScaler().fit_transform(X_train)

In [140]:
import warnings
warnings.filterwarnings('ignore')

In [141]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=89)

In [142]:
from sklearn.manifold import TSNE

tsne = TSNE(random_state=89)

X_tsne = tsne.fit_transform(X_scaled)

In [143]:
X_tsne.shape

(50, 2)

In [144]:
rfc = RandomForestClassifier(n_estimators=100, random_state=89, 
                             n_jobs=-1, oob_score=True)
rfc_param = {'max_features': [1, 2, 3],
             'min_samples_leaf': [1, 3, 5, 8],
             'max_depth': [3, 4, 5, 8, 12, 16, 20]
            }

rfc_gcv = GridSearchCV(rfc, rfc_param, n_jobs=-1, cv=skf, verbose=1)
rfc_gcv.fit(X_tsne, y_train)

rfc_gcv.best_estimator_, rfc_gcv.best_score_

Fitting 5 folds for each of 84 candidates, totalling 420 fits


(RandomForestClassifier(max_depth=4, max_features=2, n_jobs=-1, oob_score=True,
                        random_state=89),
 0.3)

In [145]:
knn_param = {'n_neighbors': range(1, 10)}
knn = KNeighborsClassifier(n_jobs=-1)

knn_gcv = GridSearchCV(knn, knn_param, cv=skf, n_jobs=-1, verbose=True)
knn_gcv.fit(X_tsne, y_train)

knn_gcv.best_params_, knn_gcv.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


({'n_neighbors': 2}, 0.25999999999999995)

In [152]:
rec_df = recommendations_df.set_index('id').drop(['mode'], axis=1)

rfc_gcv.best_estimator_.fit(X_tsne, y_train)
rec_df_scaled = StandardScaler().fit_transform(rec_df)
rec_df_scaled_tsne = tsne.fit_transform(rec_df_scaled)
y_pred = rfc_gcv.best_estimator_.predict(rec_df_scaled_tsne)

rec_df['ratings'] = y_pred
rec_df = rec_df.sort_values('ratings', ascending=False).reset_index()

rec_to_add = rec_df[rec_df['ratings']>=9]['id'].values.tolist()

In [153]:
def remove_list_duplicates(input_list):
    output_list = []
    [output_list.append(x) for x in input_list if x not in output_list]
    return output_list

In [154]:
rec_to_add = remove_list_duplicates(rec_to_add)

In [149]:
ml_playlist = sp.user_playlist_create(user=username, name='Recommendations for Playlist',  description = 'Machine Learning Recommended Tracks')

In [155]:
sp.user_playlist_add_tracks(username, ml_playlist['id'], rec_to_add);
    